In [8]:
import time
import os
import mne
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import brainflow
from brainflow.board_shim import BrainFlowInputParams, BoardShim
from brainflow.data_filter import DataFilter

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, AveragePooling2D
from tensorflow.keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
X_train_11 = np.load(r"C:\Users\Zack\OneDrive\Drexel\2021 - 2022\BMES 725\Final Project\BMES725-BCI-Motor-Imagery\numpy_data\X_train_11.npy")
X_val_11 = np.load(r"C:\Users\Zack\OneDrive\Drexel\2021 - 2022\BMES 725\Final Project\BMES725-BCI-Motor-Imagery\numpy_data\X_val_11.npy")
Y_train_11 = np.load(r"C:\Users\Zack\OneDrive\Drexel\2021 - 2022\BMES 725\Final Project\BMES725-BCI-Motor-Imagery\numpy_data\Y_train_11.npy")
Y_val_11 = np.load(r"C:\Users\Zack\OneDrive\Drexel\2021 - 2022\BMES 725\Final Project\BMES725-BCI-Motor-Imagery\numpy_data\Y_val_11.npy")

In [9]:
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   

def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('sigmoid')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

model = ShallowConvNet(nb_classes=2, Chans=8, Samples=257)
model.summary()

model.compile(loss="categorical_crossentropy", optimizer ="adam", metrics=["accuracy"])

epochs = 100
batch_size = 25
fitted_model = model.fit(X_train_11, Y_train_11, batch_size=batch_size, epochs=epochs, validation_data=(X_val_11, Y_val_11))

#score = model.evaluate(test_X, test_y, batch_size=batch_size)
#classification_accuracy = round(score[1]*100, 2)
#print("\nClassification accuracy: " + str(classification_accuracy))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 257, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 245, 40)        560       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 245, 40)        12800     
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 245, 40)        160       
_________________________________________________________________
activation (Activation)      (None, 1, 245, 40)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 31, 40)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 1, 31, 40)         0     

In [11]:
print(os.getcwd())
model_name = f"test_model.model"
model.save(model_name)
print("\nModel saved.")
print(model_name)

c:\Users\Zack\OneDrive\Drexel\2021 - 2022\BMES 725\Final Project\BMES725-BCI-Motor-Imagery
INFO:tensorflow:Assets written to: test_model.model\assets

Model saved.
test_model.model
